# 1. Aspect Extraction & Sentiment Analysis

In [1]:
import pandas as pd
import numpy as np
# from pyabsa.functional import ABSADatasetList
from pyabsa.functional import ATEPCCheckpointManager
import pickle

This script could only be used to manage NVIDIA GPUs,but no GPU found in your device


/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/neural/train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from .optimizers import Adam, linear_decay
/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/Users/xiaoxiangma/.local/lib/python3.7/site-packages/matplotlib/backend_bases.py:55: DeprecationWarning

Remote ABSADataset version: 2022.07.22 Local ABSADatasets version: None
Unknown local version for ABSADatasets, please check the latest version of ABSADatasets at https://github.com/yangheng95/ABSADatasets


In [2]:
res_reviews = pd.read_csv("Restaurant_Reviews.tsv", sep='\t')
review_list = res_reviews.Review.tolist()

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
try:
    with open('aspect_extractor.pkl', 'rb') as file:
        aspect_extractor = pickle.load(file)


except:  
    aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english')
    with open('aspect_extractor.pkl', 'wb') as file:
        pickle.dump(aspect_extractor, file)

In [4]:
# atepc_result = aspect_extractor.extract_aspect(inference_source=res_reviews.Review.tolist(),
#                                                save_result=True,
#                                                print_result=True,  # print the result
#                                                pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
#                                                )

# 2. Prepare for clustering

In [12]:
import json
  
# Opening JSON file
f = open('atepc_inference.result.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
f.close()

In [13]:
# main = pd.read_json('atepc_inference.result.json')
# main

In [224]:
main = []
for e, sentence in enumerate(data[:]):
    single = []
    if True:
        single.append(e)
        single.append(sentence['sentence'])
        single.append(sentence['tokens'])
        single.append(sentence['aspect'])
        single.append([i[0] for i in sentence['position']])
        single.append(sentence['sentiment'])
        main.append(single)
main = pd.DataFrame(main, columns =['sentence_id', 'sentence', 'tokens', 'aspect', 'aspect_loc', 'aspect_sentiment'])

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [225]:
main

,sentence_id,sentence,tokens,aspect,aspect_loc,aspect_sentiment
0,0,Wow . . . Loved this place .,"[Wow, ., ., ., Loved, this, place, .]",[place],[7],[Positive]
1,1,Crust is not good .,"[Crust, is, not, good, .]",[Crust],[1],[Negative]
2,2,Not tasty and the texture was just nasty .,"[Not, tasty, and, the, texture, was, just, nas...",[texture],[5],[Negative]
3,3,Stopped by during the late May bank holiday of...,"[Stopped, by, during, the, late, May, bank, ho...",[],[],[]
4,4,The selection on the menu was great and so wer...,"[The, selection, on, the, menu, was, great, an...","[selection, menu, prices]","[2, 5, 12]","[Neutral, Neutral, Positive]"
...,...,...,...,...,...,...
995,995,I think food should have flavor and texture an...,"[I, think, food, should, have, flavor, and, te...","[food, flavor, texture]","[3, 6, 8]","[Neutral, Negative, Negative]"
996,996,Appetite instantly gone .,"[Appetite, instantly, gone, .]",[Appetite],[1],[Negative]
997,997,Overall I was not impressed and would not go b...,"[Overall, I, was, not, impressed, and, would, ...",[],[],[]
998,998,"The whole experience was underwhelming , and I...","[The, whole, experience, was, underwhelming, ,...","[experience, Sushi]","[3, 17]","[Negative, Neutral]"


In [231]:
aspects_loc = []
for e, sentence in enumerate(data[:]):
    if sentence['aspect'] != []:
        for i in range(len(sentence['aspect'])):
            aspects_loc.append([e,sentence['position'][i][0],sentence['aspect'][i]])
aspects_loc = pd.DataFrame(aspects_loc, columns =['sentence_id', 'loc','aspect'])


In [232]:
aspects_loc

,sentence_id,loc,aspect
0,0,7,place
1,1,1,Crust
2,2,5,texture
3,4,2,selection
4,4,5,menu
...,...,...,...
1183,995,8,texture
1184,996,1,Appetite
1185,998,3,experience
1186,998,17,Sushi


# 3. Clustering: Umap & HDBscan

In [18]:
import random
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hdbscan
import umap
import tensorflow as tf

import tensorflow_hub as hub

from sentence_transformers import SentenceTransformer

from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

from tqdm.notebook import trange
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C hea

In [233]:
all_intents = list(aspects_loc['aspect'])
len(all_intents)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1188

In [20]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# model_use = hub.load(module_url)

# print(f"module {module_url} loaded")

In [21]:
model_st1 = SentenceTransformer('all-mpnet-base-v2')
model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
model_st3 = SentenceTransformer('all-distilroberta-v1')

### Embed phrases

In [22]:
def embed(model, model_type, sentences):
    if model_type == 'use':
        embeddings = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences, show_progress_bar=True)
    
    return embeddings

In [23]:
embeddings_st1 = embed(model_st1, 'sentence transformer', all_intents)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [24]:
embeddings_st2 = embed(model_st2, 'sentence transformer', all_intents)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [25]:
embeddings_st3 = embed(model_st3, 'sentence transformer', all_intents)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [26]:
embeddings = [embeddings_st1, embeddings_st2, embeddings_st3]

for embedding in embeddings:
    print(embedding.shape)

(1188, 768)
(1188, 384)
(1188, 768)


### Dimensionality reduction

In [66]:
def umap_fit(message_embeddings,
                      n_neighbors,
                      n_components, 
                      random_state = None):
    reducer = umap.UMAP(n_neighbors = n_neighbors, 
                                n_components = n_components, 
                                metric = 'cosine', 
                                random_state=random_state).fit(message_embeddings)
    return(reducer)

In [69]:
reducer = umap_fit(embeddings_st1, n_neighbors = 15, 
                                     n_components = 5,)

In [68]:
def generate_clusters(umap_reducer,
                      message_embeddings,
                      min_cluster_size,
                      min_samples = None):
    """
    Returns HDBSCAN objects after first performing dimensionality reduction using UMAP
    
    Arguments:
        message_embeddings: embeddings to use
        n_neighbors: int, UMAP hyperparameter n_neighbors
        n_components: int, UMAP hyperparameter n_components
        min_cluster_size: int, HDBSCAN hyperparameter min_cluster_size
        min_samples: int, HDBSCAN hyperparameter min_samples
        random_state: int, random seed
        
    Returns:
        clusters: HDBSCAN object of clusters
    """
    

    umap_embeddings = umap_reducer.transform(message_embeddings)
    
    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size, 
                               min_samples = min_samples,
                               metric='euclidean', 
                               gen_min_span_tree=True,
                               cluster_selection_method='eom',
                              prediction_data=True).fit(umap_embeddings)
    
    return clusters

In [71]:
clusters_default = generate_clusters(reducer, embeddings_st1, 
                                     min_cluster_size = 10,)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [72]:
clusters_default

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=True, leaf_size=40,
        match_reference_implementation=False, max_cluster_size=0,
        memory=Memory(location=None), metric='euclidean', min_cluster_size=10,
        min_samples=None, p=None, prediction_data=True)

In [73]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given clustering

    Arguments:
        clusters: HDBSCAN clustering object
        prob_threshold: float, probability threshold to use for deciding
                        what cluster labels are considered low confidence

    Returns:
        label_count: int, number of unique cluster labels, including noise
        cost: float, fraction of data points whose cluster assignment has
              a probability below cutoff threshold
    """
    
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)
    
    return label_count, cost

In [74]:
score_clusters(clusters_default)

(39, 0.07912457912457913)

In [80]:
def objective(params, embeddings, label_lower, label_upper):
    """
    Objective function for hyperopt to minimize

    Arguments:
        params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'random_state' and
               their values to use for evaluation
        embeddings: embeddings to use
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters

    Returns:
        loss: cost function result incorporating penalties for falling
              outside desired range for number of clusters
        label_count: int, number of unique cluster labels, including noise
        status: string, hypoeropt status

        """
    reducer = umap_fit(embeddings, n_neighbors = params['n_neighbors'], 
                                 n_components = params['n_components'],
                                 random_state = params['random_state'])
    
    clusters = generate_clusters(reducer, 
                                 embeddings,
                                 min_cluster_size = params['min_cluster_size'])
    
    label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
    
    #15% penalty on the cost function if outside the desired range of groups
    if (label_count < label_lower) | (label_count > label_upper):
        penalty = 0.15 
    else:
        penalty = 0
    
    loss = cost + penalty
    
    return {'loss': loss, 'label_count': label_count, 'status': STATUS_OK, 'reducer': reducer, 'HDBSCANclusters':clusters}

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
objective({
    "n_neighbors": 11,
    "n_components": 8,
    "min_cluster_size": 6,
    "random_state": 42}, embeddings_st1,  label_lower = 15, label_upper = 20
          
         )

{'loss': 0.2106060606060606,
 'label_count': 56,
 'status': 'ok',
 'reducer': UMAP(angular_rp_forest=True, metric='cosine', n_components=8, n_neighbors=11, random_state=42, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'HDBSCANclusters': HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
         approx_min_span_tree=True, cluster_selection_epsilon=0.0,
         cluster_selection_method='eom', core_dist_n_jobs=4,
         gen_min_span_tree=True, leaf_size=40,
         match_reference_implementation=False, max_cluster_size=0,
         memory=Memory(location=None), metric='euclidean', min_cluster_size=6,
         min_samples=None, p=None, prediction_data=True)}

In [82]:
def bayesian_search(embeddings, space, label_lower, label_upper, max_evals=100):
    """
    Perform bayesian search on hyperparameter space using hyperopt

    Arguments:
        embeddings: embeddings to use
        space: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', and 'random_state' and
               values that use built-in hyperopt functions to define
               search spaces for each
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters
        max_evals: int, maximum number of parameter combinations to try

    Saves the following to instance variables:
        best_params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'min_samples', and 'random_state' and
               values associated with lowest cost scenario tested
        best_clusters: HDBSCAN object associated with lowest cost scenario
                       tested
        trials: hyperopt trials object for search

        """
    
    trials = Trials()
    fmin_objective = partial(objective, 
                             embeddings=embeddings, 
                             label_lower=label_lower,
                             label_upper=label_upper)
    
    best = fmin(fmin_objective, 
                space = space, 
                algo=tpe.suggest,
                max_evals=max_evals, 
                trials=trials)

    best_params = space_eval(space, best)
    print ('best:')
    print (best_params)
    print (f"label count: {trials.best_trial['result']['label_count']}")
    

    best_reducer = umap_fit(embeddings, n_neighbors = best_params['n_neighbors'], 
                            n_components = best_params['n_components'],
                            random_state = best_params['random_state'])
    
    best_clusters = generate_clusters(best_reducer, 
                                      embeddings,
                                      min_cluster_size = best_params['min_cluster_size'])
    
    
    return best_params, best_reducer, best_clusters, trials

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [84]:
hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,50)),
    "n_components": hp.choice('n_components', range(10,50)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,16)),
    "random_state": 42
}

label_lower = 10
label_upper = 20
max_evals = 15

In [85]:
best_params_st1, best_reducer_st1, best_clusters_st1, trials_st1 = bayesian_search(embeddings_st1, 
                                                                 space=hspace, 
                                                                 label_lower=label_lower, 
                                                                 label_upper=label_upper, 
                                                                 max_evals=max_evals)

100%|██████████| 15/15 [04:31<00:00, 18.12s/trial, best loss: 0.21986531986531987]
best:
{'min_cluster_size': 9, 'n_components': 18, 'n_neighbors': 10, 'random_state': 42}
label count: 45


In [94]:
best_params_st2, best_reducer_st2, best_clusters_st2, trials_st2 = bayesian_search(embeddings_st2, 
                                                                 space=hspace, 
                                                                 label_lower=label_lower, 
                                                                 label_upper=label_upper, 
                                                                 max_evals=max_evals)

100%|██████████| 15/15 [03:54<00:00, 15.64s/trial, best loss: 0.19797979797979798]
best:
{'min_cluster_size': 3, 'n_components': 40, 'n_neighbors': 17, 'random_state': 42}
label count: 87


In [95]:
best_params_st3, best_reducer_st3, best_clusters_st3, trials_st3 = bayesian_search(embeddings_st3, 
                                                                 space=hspace, 
                                                                 label_lower=label_lower, 
                                                                 label_upper=label_upper, 
                                                                 max_evals=max_evals)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


100%|██████████| 15/15 [03:58<00:00, 15.87s/trial, best loss: 0.04040404040404041]
best:
{'min_cluster_size': 14, 'n_components': 24, 'n_neighbors': 46, 'random_state': 42}
label count: 11


{'min_cluster_size': 14, 'n_components': 24, 'n_neighbors': 46, 'random_state': 42}
label count: 11

In [98]:
def combine_results(df_ground, cluster_dict):
    """
    Returns dataframe of all documents and each model's assigned cluster

    Arguments:
        df_ground: dataframe of original documents with associated ground truth
                   labels
        cluster_dict: dict, keys as column name for specific model and value as
                      best clusters HDBSCAN object

    Returns:
        df_combined: dataframe of all documents with labels from
                     best clusters for each model

    """

    df_combined = df_ground.copy()
    
    for key, value in cluster_dict.items():
        df_combined[key] = value.labels_
    
    return df_combined

In [235]:
cluster_dict = {'label_st1': best_clusters_st1, 
                'label_st2': best_clusters_st2,
                'label_st3': best_clusters_st3}

results_df = combine_results(aspects_loc, cluster_dict)

results_df

,sentence_id,loc,aspect,label_st1,label_st2,label_st3
0,0,7,place,2,6,0
1,1,1,Crust,30,60,6
2,2,5,texture,16,67,6
3,4,2,selection,0,1,6
4,4,5,menu,10,8,4
...,...,...,...,...,...,...
1183,995,8,texture,16,67,6
1184,996,1,Appetite,21,22,6
1185,998,3,experience,18,15,7
1186,998,17,Sushi,32,82,6


In [236]:
results_df[results_df.label_st3== 1]

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sentence_id,loc,aspect,label_st1,label_st2,label_st3
10,10,1,Service,4,13,1
16,17,7,service,4,13,1
20,22,1,Service,4,13,1
42,37,2,service,4,13,1
46,39,1,Service,4,13,1
...,...,...,...,...,...,...
1108,931,12,service,4,13,1
1133,953,4,service,4,13,1
1142,961,18,service,4,13,1
1146,968,9,service,4,13,1


In [237]:
aspect_clusters = results_df.groupby('label_st3').agg(group=('aspect', ','.join)).reset_index()

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [238]:
aspect_clusters

,label_st3,group
0,-1,"hair,Waitress,tables,waiter,table,people,table..."
1,0,"place,touch,place,interior,place,place,outside..."
2,1,"Service,service,Service,service,Service,servic..."
3,2,"food,stuff,food,food,food,food,food,food,food,..."
4,3,"prices,prices,discount,value,price,prices,pric..."
5,4,"menu,menu,menu suggestions,menu,menu,menus,men..."
6,5,"server,servers,server,servers,mains,servers,Se..."
7,6,"Crust,texture,selection,pho,potatoes,fries,cra..."
8,7,"establishment,attitudes,ambience,atmosphere,ex..."
9,8,"cashier,management,business,Owner,corporation,..."


In [179]:
key_phrases = [
    'service',
    'hygeine', 
    'appetizer', 
    'desserts',
    'drinks/bar', 
    'food',
    'dishes', 
    'atmosphere', 
    'menu', 
    'price', 
    'management',
    'staff'
]

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [180]:
key_phrases_embeddings_st3 = embed(model_st3, 'sentence transformer', key_phrases)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [181]:
clusters_embedding = embed(model_st3, 'sentence transformer', aspect_clusters.group.to_list())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [140]:
from numpy.linalg import norm
def cosine_sim(A,B):
    cosine = np.dot(A,B)/(norm(A)*norm(B))
    return cosine

In [188]:
labels = []
for i in range(len(clusters_embedding)):

    labels.append([cosine_sim(phrase, clusters_embedding[i]) for phrase in key_phrases_embeddings_st3])

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [258]:
test = pd.DataFrame(labels, columns =key_phrases)
test['category'] = test.idxmax(axis=1)
test['category_strength'] = test.max(axis=1)
test = test.round(2)

In [260]:
test

,service,hygeine,appetizer,desserts,drinks/bar,food,dishes,atmosphere,menu,price,management,staff,category,category_strength
0,0.22,0.16,0.22,0.15,0.23,0.19,0.20,0.16,0.13,0.12,0.27,0.36,staff,0.36
1,0.11,0.20,0.16,0.14,0.23,0.19,0.21,0.32,0.13,0.08,0.07,0.14,atmosphere,0.32
2,0.44,0.19,0.20,0.15,0.17,0.20,0.20,0.16,0.17,0.11,0.21,0.27,service,0.44
3,0.09,0.15,0.19,0.14,0.10,0.37,0.12,0.19,0.18,0.09,0.08,0.15,food,0.37
4,0.17,0.17,0.19,0.16,0.21,0.17,0.13,0.20,0.15,0.49,0.08,0.11,price,0.49
5,0.16,0.15,0.25,0.23,0.32,0.29,0.22,0.15,0.69,0.16,0.17,0.18,menu,0.69
6,0.31,0.16,0.08,0.14,0.18,0.10,0.15,0.13,0.16,0.09,0.16,0.23,service,0.31
7,0.09,0.07,0.14,0.19,0.18,0.24,0.23,0.04,0.22,0.03,0.01,0.05,food,0.24
8,0.17,0.18,0.27,0.17,0.22,0.22,0.11,0.48,0.18,0.10,0.10,0.12,atmosphere,0.48
9,0.29,0.10,0.22,0.13,0.18,0.16,0.12,0.11,0.16,0.02,0.50,0.40,management,0.50


In [261]:
label_strength = pd.concat([aspect_clusters, test[['category','category_strength']]], axis=1)
label_strength

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,label_st3,group,category,category_strength
0,-1,"hair,Waitress,tables,waiter,table,people,table...",staff,0.36
1,0,"place,touch,place,interior,place,place,outside...",atmosphere,0.32
2,1,"Service,service,Service,service,Service,servic...",service,0.44
3,2,"food,stuff,food,food,food,food,food,food,food,...",food,0.37
4,3,"prices,prices,discount,value,price,prices,pric...",price,0.49
5,4,"menu,menu,menu suggestions,menu,menu,menus,men...",menu,0.69
6,5,"server,servers,server,servers,mains,servers,Se...",service,0.31
7,6,"Crust,texture,selection,pho,potatoes,fries,cra...",food,0.24
8,7,"establishment,attitudes,ambience,atmosphere,ex...",atmosphere,0.48
9,8,"cashier,management,business,Owner,corporation,...",management,0.50


In [262]:
aspects_details = results_df.merge(label_strength, on='label_st3',how='left')
aspects_details.drop(columns=['label_st1', 'label_st2', 'label_st3',
       'group'], inplace=True)
aspects_details

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sentence_id,loc,aspect,category,category_strength
0,0,7,place,atmosphere,0.32
1,1,1,Crust,food,0.24
2,2,5,texture,food,0.24
3,4,2,selection,food,0.24
4,4,5,menu,menu,0.69
...,...,...,...,...,...
1183,995,8,texture,food,0.24
1184,996,1,Appetite,food,0.24
1185,998,3,experience,atmosphere,0.48
1186,998,17,Sushi,food,0.24


In [263]:
category_column = aspects_details.groupby('sentence_id')['category'].apply(list)
category_strength_column = aspects_details.groupby('sentence_id')['category_strength'].apply(list)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [264]:
main['category'] = category_column
main['category_strength'] = category_strength_column

In [265]:
main

,sentence_id,sentence,tokens,aspect,aspect_loc,aspect_sentiment,category,category_strength
0,0,Wow . . . Loved this place .,"[Wow, ., ., ., Loved, this, place, .]",[place],[7],[Positive],[atmosphere],[0.32]
1,1,Crust is not good .,"[Crust, is, not, good, .]",[Crust],[1],[Negative],[food],[0.24]
2,2,Not tasty and the texture was just nasty .,"[Not, tasty, and, the, texture, was, just, nas...",[texture],[5],[Negative],[food],[0.24]
3,3,Stopped by during the late May bank holiday of...,"[Stopped, by, during, the, late, May, bank, ho...",[],[],[],NaN,NaN
4,4,The selection on the menu was great and so wer...,"[The, selection, on, the, menu, was, great, an...","[selection, menu, prices]","[2, 5, 12]","[Neutral, Neutral, Positive]","[food, menu, price]","[0.24, 0.69, 0.49]"
...,...,...,...,...,...,...,...,...
995,995,I think food should have flavor and texture an...,"[I, think, food, should, have, flavor, and, te...","[food, flavor, texture]","[3, 6, 8]","[Neutral, Negative, Negative]","[food, food, food]","[0.37, 0.24, 0.24]"
996,996,Appetite instantly gone .,"[Appetite, instantly, gone, .]",[Appetite],[1],[Negative],[food],[0.24]
997,997,Overall I was not impressed and would not go b...,"[Overall, I, was, not, impressed, and, would, ...",[],[],[],NaN,NaN
998,998,"The whole experience was underwhelming , and I...","[The, whole, experience, was, underwhelming, ,...","[experience, Sushi]","[3, 17]","[Negative, Neutral]","[atmosphere, food]","[0.48, 0.24]"


In [139]:
test_new_points = ['service', 'waitress', 'lobster']
test_new_points_embeddings_st1 = embed(model_st1, 'sentence transformer', test_new_points)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [91]:
test_new_points_embeddings_st1_umap = best_reducer_st1.transform(test_new_points_embeddings_st1)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [92]:
test_labels, strengths = hdbscan.approximate_predict(best_clusters_st1, test_new_points_embeddings_st1_umap)
test_labels

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 4,  8, 43], dtype=int32)